# __Homework 4:__ Practical analysis with BioPython

For the homework, you are going to extend the code from the analysis of our FASTQ file in lectures 8 and 9.
Recall that the FASTQ file contains reads from a real sequencing run of influenza virus HA and NA genes.

---
The __actual sequences__ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
---


__The end of NA is__ `...CACGATAGATAAATAATAGTGCACCAT`
    
__The end of HA is__ `...CCGGATTTGCATATAATGATGCACCAT`

---    

    
The __sequencing reads__ from the reverse end of the molecules (in 5'>3' orientation), so the sequencing reads are as follows:

    5'-[reverse complement of 16 X N barcode]-GCGGCCGCCT-[reverse complement of the end of HA]-3'
or

    5'-[reverse complement of 16 X N barcode]-GCGGCCGCCT-[reverse complement of the end of NA]-3'

---   
    
The reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example exercise in class, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.
This homework can be completed almost entirely by re-using code from lecture 9. You will need to set up your analysis to do the following:
 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 4. Determine the number and distribution of barcodes for HA and NA separately.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

1. How many reads map to HA, and how many reads map to NA?

In [ ]:
By 상헌
#By 상헌

# Define the function to extract barcode and identify the gene
def extract_barcode_and_gene(read):
    # Define the patterns for HA and NA sequences
    ha_pattern = re.compile(r'CCGGATTTGCATATAATGATGCACCAT')  # Pattern for HA gene
    na_pattern = re.compile(r'CACGATAGATAAATAATAGTGCACCAT')  # Pattern for NA gene

    # Get the reverse complement of the read
    # reverse_read = reverse_complement(read)
    reverse_read = str(read.reverse_complement())

    # Search for the gene patterns in the reverse complement
    ha_match = ha_pattern.search(reverse_read)
    na_match = na_pattern.search(reverse_read)

    # Check if the read matches HA or NA gene, extract barcode, and count
    if ha_match:
        if check_barcode(reverse_read):
            barcode = extract_barcode(reverse_read)
            return 'HA', barcode
        else:
            return 'HA', None
    elif na_match:
        if check_barcode(reverse_read):
            barcode = extract_barcode(reverse_read)
            return 'NA', barcode
        else:
            return 'NA', None
    else:
        return None, None  # Return None if neither HA nor NA gene is found

# Define a function to extract the barcode
def extract_barcode(read, bclen=16):
    barcode = read[-bclen:]  # Extract the barcode
    return barcode

def check_barcode(read,  bclen=16, upstream='AGGCGGCCGC'):
    barcode_re = re.compile(upstream + "(?P<barcode>[ATCG]{" + str(bclen) + "})$")
    
    match = barcode_re.search(read)

    if match:
        barcode = match.group("barcode")
    else:
        barcode = None
    
    return barcode

# Create dictionaries to store barcode counts for HA and NA
ha_barcodes = {}
na_barcodes = {}

# Iterate through the FASTQ file or sequence reads
total_ha, total_na = 0, 0
invalid_ha, invalid_na = 0, 0
invalid_all = 0
for seq in seqreads_Seq:
    gene, barcode = extract_barcode_and_gene(seq)
    if gene == 'HA':
        # print(str(seq.reverse_complement())[-16:])
        if barcode is None:
            invalid_ha += 1
        elif barcode in ha_barcodes:
            ha_barcodes[barcode] += 1
            total_ha += 1
        else:
            ha_barcodes[barcode] = 1
            total_ha += 1
    elif gene == 'NA':
        # print(str(seq.reverse_complement())[-16:])
        if barcode is None:
            invalid_na += 1
        elif barcode in na_barcodes:
            na_barcodes[barcode] += 1
            total_na += 1
        else:
            na_barcodes[barcode] = 1
            total_na += 1
    else:
        invalid_all += 1

# Display barcode counts for HA and NA genes
print("Barcode counts for HA gene:")
print(ha_barcodes)

_sorted_ha_barcode = sorted(ha_barcodes, key=lambda x: ha_barcodes[x])[::-1]
sorted_ha_barcode = []
for x in _sorted_ha_barcode:
    sorted_ha_barcode.append((x, ha_barcodes[x]))
print(sorted_ha_barcode)



print(f'total ha: {total_ha}')
print(f'invalid ha: {invalid_ha}')

print("\nBarcode counts for NA gene:")
print(na_barcodes)
print(f'total na: {total_na}')
print(f'invalid na: {invalid_na}')

print(f'invalid all: {invalid_all}')

print(total_ha + total_na + invalid_ha + invalid_na + invalid_all)

In [ ]:
# load necessary packages
import re
import Bio.SeqIO
from Bio.Seq import Seq

In [ ]:
# HA read
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq'))

seqreads_Seq = []
for seqrecord in seqreads:
    seqreads_Seq.append(seqrecord.seq)

def read_barcode(seqread, upstream='CACGATAGATAAATAATAGTGCACCAT'):

    
    sequence = seqread.reverse_complement()
    sequence_str = str(sequence)
    
    barcode_re = re.compile(upstream + "(?P<barcode>[ATCG])")
    
    match = barcode_re.search(sequence_str)

    if match:
        barcode = match.group("barcode")
    else:
        barcode = None
    
    return barcode


barcode_counts = {}
n_invalid = 0

for seq in seqreads_Seq: # iterate through all reads
    barcode = read_barcode(seq, upstream='CACGATAGATAAATAATAGTGCACCAT')
    if barcode: # if there is a valid barcode, add it to the dictionary
        if barcode in barcode_counts:
            barcode_counts[barcode] += 1
        else:
            barcode_counts[barcode] = 1
    else:
        n_invalid += 1

print(f"There were {len(seqreads_Seq)} total sequence reads parsed, and {n_invalid} lacked a valid barcode.")
print(barcode_counts)



There were 10000 total sequence reads parsed, and 5878 lacked a valid barcode.
{'A': 4119, 'G': 2, 'T': 1}


In [5]:
# run this code chunk...
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq'))

seqreads_Seq = []
for seqrecord in seqreads:
    seqreads_Seq.append(seqrecord.seq)

def read_barcode(seqread, bclen, upstream='AGGCGGCCGC'):

    
    sequence = seqread.reverse_complement()
    sequence_str = str(sequence)
    
    barcode_re = re.compile(upstream + "(?P<barcode>[ATCG]{" + str(bclen) + "})$") #regular expression
    
    match = barcode_re.search(sequence_str)

    if match:
        barcode = match.group("barcode")
    else:
        barcode = None
    
    return barcode


barcode_counts = {}
n_invalid = 0

for seq in seqreads_Seq: # iterate through all reads
    barcode = read_barcode(seq, bclen = 16, upstream='AGGCGGCCGC')
    if barcode: # if there is a valid barcode, add it to the dictionary
        if barcode in barcode_counts:
            barcode_counts[barcode] += 1
        else:
            barcode_counts[barcode] = 1
    else:
        n_invalid += 1

print(len(seqreads_Seq))
print(n_invalid)




10000
569


In [6]:
# run this code chunk...
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq'))

seqreads_Seq = []
for seqrecord in seqreads:
    seqreads_Seq.append(seqrecord.seq)

def read_barcode(seqread):

    
    sequence = seqread.reverse_complement()
    sequence_str = str(sequence)
    
    barcode_re = re.compile(r'CCGGATTTGCATATAATGATGCACCAT')
    
    match = barcode_re.search(sequence_str)

    if match:
        barcode = match.group("barcode")
    else:
        barcode = None
    
    return barcode


barcode_counts = {}
n_invalid = 0

for seq in seqreads_Seq: # iterate through all reads
    barcode = read_barcode(r'CCGGATTTGCATATAATGATGCACCAT')
    if barcode: # if there is a valid barcode, add it to the dictionary
        if barcode in barcode_counts:
            barcode_counts[barcode] += 1
        else:
            barcode_counts[barcode] = 1
    else:
        n_invalid += 1

print(f"There were {len(seqreads_Seq)} total sequence reads parsed, and {n_invalid} lacked a valid barcode.")


print(barcode_counts)

AttributeError: 'str' object has no attribute 'reverse_complement'

In [4]:
# run this code chunk...
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq'))

seqreads_Seq = []
for seqrecord in seqreads:
    seqreads_Seq.append(seqrecord.seq)

def read_barcode(seqread, bclen, upstream='AGGCGGCCGC'):

    
    sequence = seqread.reverse_complement()
    sequence_str = str(sequence)
    
    barcode_re = re.compile(upstream + "(?P<barcode>[ATCG]{" + str(bclen) + "})$")
    
    match = barcode_re.search(sequence_str)

    if match:
        barcode = match.group("barcode")
    else:
        barcode = None
    
    return barcode


barcode_counts = {}
n_invalid = 0

for seq in seqreads_Seq: # iterate through all reads
    barcode = read_barcode(seq, bclen = 16, upstream='AGGCGGCCGC')
    if barcode: # if there is a valid barcode, add it to the dictionary
        if barcode in barcode_counts:
            barcode_counts[barcode] += 1
        else:
            barcode_counts[barcode] = 1
    else:
        n_invalid += 1

print(f"There were {len(seqreads_Seq)} total sequence reads parsed, and {n_invalid} lacked a valid barcode.")


print(barcode_counts)

There were 10000 total sequence reads parsed, and 569 lacked a valid barcode.
{'ACTAAATAACTTAAGC': 63, 'AACCGTGACCAGGAAG': 70, 'CCACATTCATCGCTGA': 30, 'TTATCGTCTCCCATAT': 78, 'CTACCCGTTTCCCAAC': 124, 'CATACCAGTCATCCCT': 29, 'ACTTACGTATAAGTCA': 56, 'GCTACTACTATACCAT': 127, 'CGTCTTCCATCCCCAT': 136, 'GCGAGTCCACCAATTG': 97, 'GTTACCCACAGTCCGC': 64, 'ATATAAATAAGCTATG': 51, 'CACCACACAAGGATGT': 43, 'TCTTTTCTTATTGAGG': 74, 'CTTAACCTTCCGACAA': 57, 'ATAAACGTTCCCCAAG': 111, 'TCATACATCACACTTA': 48, 'GTACCCTCCGTGAATC': 100, 'ACTCCACGCTACCACG': 34, 'GCACTCCTCAACCCTT': 48, 'CCGCTCCCTGCTGTCC': 43, 'ACCCGTGAACCAAAGC': 1, 'AATCGTACATCCCCAC': 127, 'CCACCAGCCATACGGC': 93, 'AAACGTAGCGATAACT': 62, 'TGGACTCCACACCCCG': 79, 'TCACGTCCCATATTAC': 10, 'TCAAGAAGCCTTGGAG': 152, 'TTTCTCCCGCATCTCG': 105, 'AATCAATACGACCTGT': 74, 'TGACAATACCCTACTC': 48, 'CCCGACCCGACATTAA': 158, 'ACGAGAGGTCGACTCG': 61, 'TTCGACTTCCTAGTAC': 87, 'TTTCTCCCCCATCTCG': 1, 'CATTCACTACTCCCAA': 94, 'TCACCAGTGATCCCTT': 67, 'ACCACCGCCACACCCT': 47, 'A

In [2]:
import re
import Bio.SeqIO
from Bio.Seq import Seq


In [ ]:
# your code here...

2. How many HA sequences did not have a valid barcode? Also anwer the same question for NA.

In [ ]:
# your code here...

3. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

    _Hint: you will need to find the key associated with the maximum value in your dictionary. There are many ways to do this._

In [ ]:
# your code here...

In [5]:
# load necessary packages
import re
import Bio.SeqIO
from Bio.Seq import Seq

#read fastq and create into list
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq'))

seqreads_Seq = []
for seqrecord in seqreads:
    seqreads_Seq.append(seqrecord.seq)

In [ ]:
# Define the function to extract barcode and identify the gene
def extract_barcode_and_gene(seqread):
    # Define the patterns for HA and NA sequences
    ha_pattern = re.compile(r'CCGGATTTGCATATAATGATGCACCAT')  # Pattern for HA gene
    na_pattern = re.compile(r'CACGATAGATAAATAATAGTGCACCAT')  # Pattern for NA gene

    # Get the reverse complement of the read
    sequence = seqread.reverse_complement()
    sequence_str = str(sequence)
    

    # Search for the gene patterns in the reverse complement
    ha_match = ha_pattern.search(sequence_str)
    na_match = na_pattern.search(sequence_str)

    # Check if the read matches HA or NA gene, extract barcode, and count
    if ha_match:
        barcode = extract_barcode(sequence_str, ha_match.end())
        return 'HA', barcode
    elif na_match:
        barcode = extract_barcode(sequence_str, na_match.end())
        return 'NA', barcode
    else:
        return None, None  # Return None if neither HA nor NA gene is found

# Define a function to extract the barcode
def extract_barcode(seqread, start):
    barcode_length = 16
    barcode = read[start - barcode_length: start]  # Extract the barcode
    return barcode

# Create dictionaries to store barcode counts for HA and NA
ha_barcodes = {}
na_barcodes = {}

# Iterate through the FASTQ file or sequence reads
for seq in seqreads_Seq:
    gene, barcode = extract_barcode_and_gene(seq)
    if gene == 'HA':
        if barcode in ha_barcodes:
            ha_barcodes[barcode] += 1
        else:
            ha_barcodes[barcode] = 1
    elif gene == 'NA':
        if barcode in na_barcodes:
            na_barcodes[barcode] += 1
        else:
            na_barcodes[barcode] = 1

# Display barcode counts for HA and NA genes
print("Barcode counts for HA gene:")
print(ha_barcodes)

print("\nBarcode counts for NA gene:")
print(na_barcodes)

: 